# Module Feedback

This system records the responses of students on their learning experience at university.

![rel](https://sqlzoo.net/w/images/7/71/Module-feedback.png)

In [1]:
library(tidyverse)
library(DBI)
library(getPass)
drv <- switch(Sys.info()['sysname'],
             Windows="PostgreSQL Unicode(x64)",
             Darwin="/usr/local/lib/psqlodbcw.so",
             Linux="PostgreSQL")
con <- dbConnect(
  odbc::odbc(),
  driver = drv,
  Server = "localhost",
  Database = "sqlzoo",
  UID = "postgres",
  PWD = getPass("Password?"),
  Port = 5432
)
options(repr.matrix.max.rows=20)

-- Attaching packages --------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.0     v purrr   0.3.4
v tibble  3.0.1     v dplyr   0.8.5
v tidyr   1.0.2     v stringr 1.4.0
v readr   1.3.1     v forcats 0.5.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



Password? ·········


## 1. Find the student name from a matriculation number

**Find the name of the student with number 50200100**

In [2]:
ins_mod <- dbReadTable(con, 'INS_MOD')
ins_prs <- dbReadTable(con, 'INS_PRS')
cam_smo <- dbReadTable(con, 'CAM_SMO')
ins_res <- dbReadTable(con, 'INS_RES')
ins_que <- dbReadTable(con, 'INS_QUE')
ins_spr <- dbReadTable(con, 'INS_SPR')
ins_cat <- dbReadTable(con, 'INS_CAT')

In [3]:
ins_spr %>% 
    filter(SPR_CODE=='50200100') %>%
    select(SPR_FNM1, SPR_SURN)

SPR_FNM1,SPR_SURN
<chr>,<chr>
Tom,Cotton


## 2. Find the modules studied by a student

**Show the module code and module name for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [4]:
# AYR_CODE length 7, so read as '2016/7 '
# should trim it before processing
ins_mod %>% 
    inner_join(
        cam_smo %>%
            filter(SPR_CODE=='50200100' &
                   str_starts(AYR_CODE, '2016/7') &
                   PSL_CODE=='TR1'),
        by=c(MOD_CODE="MOD_CODE")
    ) %>%
    select(MOD_CODE, MOD_NAME)    

MOD_CODE,MOD_NAME
<chr>,<chr>
CSN08101,Systems and Services
INF08104,Database Systems
SET08108,Software Development 2


## 3. Find the modules and module leader studied by a student

**Show the module code and module name and details of the module leader for modules studied by the student with number 50200100 in session 2016/7 TR1**

In [5]:
cam_smo %>% 
    filter(SPR_CODE=='50200100' &
           str_starts(AYR_CODE, '2016/7') &
           PSL_CODE=='TR1') %>%
    inner_join(ins_mod, by=c(MOD_CODE="MOD_CODE")) %>%
    inner_join(ins_prs, by=c(PRS_CODE="PRS_CODE")) %>%
    select(MOD_CODE, MOD_NAME, PRS_CODE, PRS_FNM1, PRS_SURN)

MOD_CODE,MOD_NAME,PRS_CODE,PRS_FNM1,PRS_SURN
<chr>,<chr>,<chr>,<chr>,<chr>
CSN08101,Systems and Services,40000008,James,Jackson
INF08104,Database Systems,40000036,Andrew,Cumming
SET08108,Software Development 2,40000408,Neil,Urquhart


## 4. Show the scores for module SET08108

**Show the Percentage of students who gave 4 or 5 to module SET08108 in session 2016/7 TR1**

(note that this is not real data, these responses were randomly generated)

In [6]:
ins_res %>% 
    filter(MOD_CODE=='SET08108' &
           str_starts(AYR_CODE, '2016/7') & 
           PSL_CODE=='TR1') %>%
    inner_join(ins_que, by=c(QUE_CODE="QUE_CODE")) %>%
    inner_join(ins_cat, by=c(CAT_CODE="CAT_CODE")) %>%
    mutate(score=floor(RES_VALU/4)) %>%
    group_by(QUE_CODE, QUE_TEXT, CAT_NAME) %>%
    summarise(pct=round(100*sum(score)/n()))

QUE_CODE,QUE_TEXT,CAT_NAME,pct
<chr>,<chr>,<chr>,<dbl>
1.1,Staff are good at explaining things.,Learning and Teaching,89
1.2,Staff made the subject interesting.,Learning and Teaching,82
1.3,The module was intellectually stimulating.,Learning and Teaching,82
1.4,The aims and objectives were clearly stated.,Learning and Teaching,89
1.5,The module was well-organised and ran smoothly.,Learning and Teaching,78
1.6,The pace was appropriate.,Learning and Teaching,80
1.7,The level was appropriate.,Learning and Teaching,82
1.8,The workload was managable.,Learning and Teaching,78
1.9,I was able to contact module staff when I needed to.,Learning and Teaching,76


## 5. Show the frequency chart for module SET08108 for question 4.1

**For each response 1-5 show the number of students who gave that response (Module SET08108, 2016/7, TR1)**

(note that this is not real data, these responses were randomly generated)

In [7]:
ins_res %>%
    filter(MOD_CODE=='SET08108' &
           str_starts(AYR_CODE, '2016/7') & 
           PSL_CODE=='TR1' &
           QUE_CODE=='4.1') %>%
    group_by(RES_VALU) %>%
    summarise(n=n())

RES_VALU,n
<int>,<int>
2,6
4,10
5,39


In [8]:
dbDisconnect(con)